In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from PIL import Image
import keras

from keras.layers import InputLayer, Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import InputLayer, Dense
import pathlib

In [3]:

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
print(data_dir)

In [ ]:
roses = list(data_dir.glob('daisy/*'))
Image.open(roses[0])

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
     

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# from tensorflow.keras.applications import ResNet50V2


In [ ]:
pre_trained = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(180, 180, 3),
    pooling='avg',
    name="resnet50",
)

for layer in pre_trained.layers:
        layer.trainable=False


transfer_model = Sequential([
        InputLayer(shape=(180, 180, 3)),
        pre_trained,

        Flatten(),
        Dense(512, activation='relu'),
        Dense(5, activation='softmax')

])

transfer_model.summary()


In [ ]:
transfer_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs=10
history = transfer_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
Image.open(roses[0])

In [ ]:
import cv2
image=cv2.imread(str(roses[0]))
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)


In [ ]:
pred=transfer_model.predict(image)
print(pred)

In [ ]:
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)